In [5202]:
import numpy as np
import math
import random

In [5203]:
# def generateKeyMatrix(n, modulo):
#     while True:
#         key_matrix = np.random.randint(1, modulo, size=(n, n))
#         try:
#             inverse_key_matrix = inverse(key_matrix, modulo)
#             return key_matrix, inverse_key_matrix
#         except ValueError:
#             continue  # Regenerate if not invertible

In [5204]:

def generateKeyMatrix(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        # Compute determinant mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")

In [5205]:
def inverse_2x2_mod(matrix, modulo):
    # Compute modular inverse for a 2x2 matrix using integer arithmetic only
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    # Adjugate matrix
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo

In [5206]:
def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range
    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]
    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T
    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

In [5207]:
def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    return ''.join(plaintext_chars)

In [5208]:

# Define the alphabet, modulo, and key matrix
modulo = 59
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
print("Alphabet Length:", len(alphabet))

key_matrix = np.array([
    [3, 2],
    [5, 7]
])

# Compute inverse key matrix dynamically
inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)

# Function to generate a random string of length l from the alphabet
def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))

# Encrypt and decrypt test
def test_encryption_decryption(l):
    plaintext = generate_random_string(alphabet, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
    return plaintext, decrypted_text, plaintext == decrypted_text

# Run 50 tests
for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)  # Change 20 to desired string length
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Alphabet Length: 59
Test 1: True
Test 2: True
Test 3: True
Test 4: True
Test 5: True
Test 6: True
Test 7: True
Test 8: True
Test 9: True
Test 10: True
Test 11: True
Test 12: True
Test 13: True
Test 14: True
Test 15: True
Test 16: True
Test 17: True
Test 18: True
Test 19: True
Test 20: True
Test 21: True
Test 22: True
Test 23: True
Test 24: True
Test 25: True
Test 26: True
Test 27: True
Test 28: True
Test 29: True
Test 30: True
Test 31: True
Test 32: True
Test 33: True
Test 34: True
Test 35: True
Test 36: True
Test 37: True
Test 38: True
Test 39: True
Test 40: True
Test 41: True
Test 42: True
Test 43: True
Test 44: True
Test 45: True
Test 46: True
Test 47: True
Test 48: True
Test 49: True
Test 50: True


In [5209]:
modulo = 59  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
print("Alphabet Length:", len(alphabet))
key_matrix = np.array([
    [3, 2],
    [5, 7]
])

print(len(alphabet))
plaintext = "Hello My name is Nic"
# Compute inverse key matrix dynamically
inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")
print(f"Condition: {np.linalg.cond(key_matrix)}")


Alphabet Length: 59
59
Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 5 56]
 [40 12]]
Original Text: Hello My name is Nic
Ciphertext: WJIfvhS?&CkqRqNGF?oM
Decrypted Text: Hello My name is Nic
Condition: 7.780565547548909


In [5210]:
modulo = 59  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!&'\n"
print("Alphabet Length:", len(alphabet))
plaintext = """Hello My name is Nic Hello My name Hello My 
name Hello My name Hello My name Hello My name 
name Hello My name Hello My name Hello My name 
Ner gonna say gobyrsqd aqq Hello check xxxxxxxxxxxx ga zz ' 
"""
key_matrix = np.array([
    [3, 2],
    [5, 7]
])

# Compute inverse key matrix dynamically
inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print()
print(f"Ciphertext: {ciphertext}")
print()
print(f"Decrypted Text: {decrypted_text}")
print()
print(f"Condition: {np.linalg.cond(key_matrix)}")

Alphabet Length: 59
Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 5 56]
 [40 12]
 [52 14]
 [46 55]
 [14 52]
 [ 3 49]
 [18 24]
 [51 23]
 [56 46]
 [52 14]
 [46 55]
 [14 52]
 [ 3 49]
 [18 24]
 [16 32]
 [36 42]
 [17 42]
 [22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [16 32]
 [36 42]
 [17 42]
 [22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [23 27]
 [58 38]
 [43 12]
 [21  1]
 [51 23]
 [ 8 37]
 [ 1  8]
 [43 12]
 [56 35]
 [ 0 20]
 [39 42]
 [14 37]
 [44 11]
 [53 43]
 [22  9]
 [ 8 31]
 [47 33]
 [32 17]
 [28 51]
 [35 13]
 [ 9 57]
 [ 9 57]
 [ 9 57]
 [ 9 57]
 [ 9 57]
 [ 9 57]
 [43 12]
 [ 5 22]
 [19 22]
 [34

In [5211]:
modulo = 59  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?'\n"
print("Alphabet Length:", len(alphabet))

plaintext = """
A full commitment's what I'm thinkin' of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching, but you're too shy to say it
"""

key_matrix = np.array([
    [3, 2],
    [5, 7]
])

# Compute inverse key matrix dynamically
inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print()
print(f"Ciphertext: {ciphertext}")
print()
print(f"Decrypted Text: {decrypted_text}")
print(f"Condition: {np.linalg.cond(key_matrix)}")


Alphabet Length: 59
Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[56 54]
 [41  5]
 [35 17]
 [38 18]
 [46  7]
 [13 43]
 [15 13]
 [56 46]
 [30 38]
 [23  3]
 [16  6]
 [33 52]
 [ 3 58]
 [20 26]
 [41 23]
 [24 43]
 [ 3 30]
 [58  5]
 [54  4]
 [ 0  9]
 [32 30]
 [34 46]
 [ 6  4]
 [47 48]
 [35 17]
 [47  5]
 [25 10]
 [43 12]
 [ 3 52]
 [10 44]
 [49 49]
 [ 0 53]
 [ 2 43]
 [19 53]
 [31 29]
 [40 14]
 [ 0  9]
 [24 43]
 [58 38]
 [43 12]
 [ 2 49]
 [13 51]
 [49 33]
 [49  7]
 [ 3 58]
 [19  9]
 [18 55]
 [ 5 22]
 [18 22]
 [ 8 31]
 [20 20]
 [35 50]
 [45 19]
 [39  5]
 [54 21]
 [11 20]
 [41  5]
 [32  6]
 [ 2 10]
 [ 4  6]
 [ 9 37]
 [33 43]
 [10 53]
 [55 54]
 [32 28]
 [17 42]
 [53 58]
 [ 6  4]
 [39 31]
 [29  1]
 [40 51]
 [10 53]
 [57 44]
 [23 27]
 [ 7  7]
 [58 38]
 [43 12]
 [21  1]
 [51 23]
 [43 12]
 [19 27]
 [17 42]
 [53 58]
 [ 6  4]
 [43 45]
 [23 27]
 [ 7  7]
 [58 38]
 [43 12]
 [21  1]
 [51 23]
 [53 47]
 [ 3 52]
 [20 20]
 [35 50]
 [37 50]
 [39  5]
 [56 11]
 [40 39]
 [24

In [5212]:
modulo = 59  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?\n"
print("Alphabet Length:", len(alphabet))

plaintext = "Hello My name is Nic"
key_matrix, inverse_key_matrix = generateKeyMatrix(2, modulo)

# Compute inverse key matrix dynamically

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")
print(f"Condition: {np.linalg.cond(key_matrix)}")

Alphabet Length: 58
Key Matrix:
[[ 1  5]
 [18 37]]
Inverse Key Matrix:
[[16  9]
 [56 10]]
Ciphertext Matrix:
[[39 56]
 [45 29]
 [ 5 48]
 [26  1]
 [11 19]
 [39 45]
 [54 45]
 [18 57]
 [58  1]
 [56 55]]
Original Text: Hello My name is Nic
Ciphertext: n?tdFwaBLTnt,tS
B?!
Decrypted Text: Hello My name iswatC
Condition: 32.403101020365355


In [5213]:

modulo = 59  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
print("Alphabet Length:", len(alphabet))

plaintext = """
A full commitment's what I'm thinkin' of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching, but you're too shy to say it
Inside, we both know what's been going on
We know the game and we're gonna play it
And if you ask me how I'm feeling
Don't tell me you're too blind to see
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
"""
key_matrix, inverse_key_matrix = generateKeyMatrix(2, modulo)

# Compute inverse key matrix dynamically

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print()
print(f"Ciphertext: {ciphertext}")
print()
print(f"Decrypted Text: {decrypted_text}")
print()
print(f"Condition: {np.linalg.cond(key_matrix)}")

Alphabet Length: 59
Failed Iteration
Key Matrix:
[[49  3]
 [39 57]]
Inverse Key Matrix:
[[28 42]
 [15 22]]
Ciphertext Matrix:
[[10 20]
 [45 19]
 [ 5  9]
 [22 41]
 [17  9]
 [29 49]
 [31 56]
 [ 5  6]
 [40 15]
 [11 19]
 [32 36]
 [52 39]
 [35  6]
 [12 21]
 [ 3 37]
 [13  9]
 [37 38]
 [ 2  1]
 [47 23]
 [23 31]
 [33 52]
 [37 44]
 [33 52]
 [12 28]
 [40 15]
 [48 17]
 [12 18]
 [28 50]
 [ 8 39]
 [11 19]
 [55  2]
 [ 9  9]
 [30 29]
 [55  5]
 [13  1]
 [ 3 37]
 [ 6 22]
 [48 17]
 [44 42]
 [34  4]
 [57 11]
 [26 54]
 [ 1 58]
 [11 50]
 [22 27]
 [14 25]
 [53 43]
 [36 12]
 [43 40]
 [33 52]
 [51 15]
 [39 48]
 [35  6]
 [12 21]
 [16 28]
 [47 34]
 [13  9]
 [31 11]
 [ 1 36]
 [39 13]
 [14 25]
 [52 14]
 [25 46]
 [43 40]
 [33 52]
 [31 48]
 [51 47]
 [ 6 22]
 [49 33]
 [34 51]
 [ 2 12]
 [19 34]
 [33  3]
 [21 39]
 [36 47]
 [22 27]
 [14 25]
 [18  0]
 [33  3]
 [43 40]
 [33 52]
 [31 48]
 [ 0  8]
 [19 34]
 [33  3]
 [21 39]
 [36 47]
 [22 27]
 [14 25]
 [15 26]
 [ 1 58]
 [33 41]
 [50 38]
 [ 7 48]
 [50 24]
 [49 53]
 [18 14]
 

In [5214]:
def inverse_nxn_mod(matrix, modulo):
    # Convert to integers and work on a copy
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    # Create identity matrix
    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo

In [5215]:
def generateKeyMatrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        # Compute determinant mod
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # Not invertible, try again
            pass

In [5216]:
def encryptNN(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range
    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]
    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T
    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

In [5217]:
def decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    return ''.join(plaintext_chars)

In [5218]:

# Define the alphabet, modulo, and key matrix
modulo = 59
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
print("Alphabet Length:", len(alphabet))
keyMatrixSize = 7

key_matrix, inverse_key_matrix = generateKeyMatrix_nxn(keyMatrixSize, modulo)

# Function to generate a random string of length l from the alphabet
def generate_random_string(alphabet, l):
    while True:
        generated = ''.join(random.choice(alphabet) for _ in range(l))
        if generated[-1] != 'A':  # Ensure the string does not end with 'A'
            return generated

# Encrypt and decrypt test
def test_encryption_decryption(l):
    plaintext = generate_random_string(alphabet, l)
    ciphertext, ciphertext_matrix = encryptNN(plaintext, key_matrix, modulo, alphabet)
    decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)

    # Count trailing 'A' characters in the decrypted text
    trailing_a_count = len(decrypted_text) - len(decrypted_text.rstrip('A'))

    return plaintext, decrypted_text, trailing_a_count, plaintext == decrypted_text

stringLength = 100
testLength = 50
correct = 0
# Run 50 tests
for idx in range(1, testLength + 1):
    plaintext, decrypted_text, trailing_a_count, result = test_encryption_decryption(stringLength)
    if result:
        print(f"Test {idx}: True")
        correct += 1
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f" Decrypted: {decrypted_text}")
    print(f" Trailing 'A' count: {trailing_a_count}")

print()
print(f"Correct: {correct}/{testLength}")
print("Plain Text Length: ", stringLength)
print("Key matrix Size  : ", keyMatrixSize)
print("Alphabet Length  : ", len(alphabet))
print(f"Plain Text Length / {keyMatrixSize}: ", stringLength/keyMatrixSize)
print(f"Plain Text Length % {keyMatrixSize}: ", stringLength%keyMatrixSize)
print(f"Plain Text Length % 59: ", stringLength%59)
print(f"Test Plaintext Length % 59 - Length %{keyMatrixSize}: ", stringLength%59 - stringLength%keyMatrixSize)
print("TEST stringLength % keyMatrixSize: ", stringLength%keyMatrixSize)
print("TEST keyMatrixSize - stringLength % keyMatrixSize: ", keyMatrixSize - stringLength%keyMatrixSize)

Alphabet Length: 59
Test 1: False
  Original: TDfaQo  LNsv?JIC ZygZReeXRwuHee&O,vruA FoV HpW
TejJgy?P!xXgK&HZ! hb i&EvZd.hIXVreyYUmHAFoumGpvhGO hP
 Decrypted: TDfaQo  LNsv?JIC ZygZReeXRwuHee&O,vruA FoV HpW
TejJgy?P!xXgK&HZ! hb i&EvZd.hIXVreyYUmHAFoumGpvhGO hPAAAAA
 Trailing 'A' count: 5
Test 2: False
  Original: AiCt&LevxelJ!cNQRhOUCVJgsY!kGtyE,mKGZqlZmoFZDajWooYkjwlZWiOKarb.BMPzBGAxTk&.,tq?CdFMCql
yTVtmUxRowNo
 Decrypted: AiCt&LevxelJ!cNQRhOUCVJgsY!kGtyE,mKGZqlZmoFZDajWooYkjwlZWiOKarb.BMPzBGAxTk&.,tq?CdFMCql
yTVtmUxRowNoAAAAA
 Trailing 'A' count: 5
Test 3: False
  Original: ImDabhDrBNsYYPLiLihTpVjBFsPyqxeBCatdwSBWQCmY!dMnvTpzh?vUehQST,cui jy RkilukKtMsiNyXBgbXNRcs&rnie MfZ
 Decrypted: ImDabhDrBNsYYPLiLihTpVjBFsPyqxeBCatdwSBWQCmY!dMnvTpzh?vUehQST,cui jy RkilukKtMsiNyXBgbXNRcs&rnie MfZAAAAA
 Trailing 'A' count: 5
Test 4: False
  Original: mGRfAmHYHe&ifJRHUpUKUMukZ.lb&siaDkNJEyz?aisJHNP.hGNUVeFPuU.A.neSKnZHVT!qM!koMc?aVnudHIXbJNKGHTfKpBsv
 Decrypted: mGRfAmHYHe&ifJRHUpUKUMukZ.lb&siaDkNJE

In [5219]:
modulo = 59  # Adjust modulo to match extended alphabet size
keyMatrixSize = 2
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!'?\n"
print("Alphabet Length:", len(alphabet))

plaintext = "Hello My name is Nic"
key_matrix, inverse_key_matrix = generateKeyMatrix_nxn(keyMatrixSize, modulo)
print("Plain Text Length:", len(plaintext))
print(f"Plain Text Length / {keyMatrixSize}:", len(plaintext)/keyMatrixSize)
print(f"Plain Text Length % {keyMatrixSize}:", len(plaintext)%keyMatrixSize)
print(f"Plain Text Length % 59:", len(plaintext)%59)
print(f"Test Plaintext Length % 59 - Length %{keyMatrixSize}:", len(plaintext)%59 - len(plaintext)%keyMatrixSize)
# Compute inverse key matrix dynamically

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Alphabet Length: 59
Plain Text Length: 20
Plain Text Length / 2: 10.0
Plain Text Length % 2: 0
Plain Text Length % 59: 20
Test Plaintext Length % 59 - Length %2: 20
Key Matrix:
[[55 51]
 [18 28]]
Inverse Key Matrix:
[[23 15]
 [40 22]]
Original Text: Hello My name is Nic
Ciphertext: bWcyO YXLWF?,xrPqC.n
Decrypted Text: Hello My name is Nic


In [5220]:
modulo = 59  # Adjust modulo to match extended alphabet size\
keyMatrixSize = 6
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?'\n"
print("Alphabet Length:", len(alphabet))

plaintext = "Hello My name is Nic"
key_matrix, inverse_key_matrix = generateKeyMatrix_nxn(keyMatrixSize, modulo)
print("Plain Text Length:", len(plaintext))
print(f"Plain Text Length / {keyMatrixSize}:", len(plaintext)/keyMatrixSize)
print(f"Plain Text Length % {keyMatrixSize}:", len(plaintext)%keyMatrixSize)
print(f"Plain Text Length % 59:", len(plaintext)%59)
print(f"Test Plaintext Length % 59 - Length %{keyMatrixSize}:", len(plaintext)%59 - len(plaintext)%keyMatrixSize)

# Compute inverse key matrix dynamically


# Encrypt the plaintext
ciphertext, ciphertext_matrix = encryptNN(plaintext, key_matrix, modulo, alphabet)
print("Plain Text Length:", len(plaintext))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")
# Decrypt the ciphertext
decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Alphabet Length: 59
Plain Text Length: 20
Plain Text Length / 6: 3.3333333333333335
Plain Text Length % 6: 2
Plain Text Length % 59: 20
Test Plaintext Length % 59 - Length %6: 18
Plain Text Length: 20
Ciphertext Length: 24
Key Matrix:
[[ 9 28 50 49 37 23]
 [20  4  9 35 50 18]
 [48 30 37  2 30 45]
 [18 43 12 24 12 50]
 [ 2 31 15 20  1 17]
 [13 16  9 57 52 28]]
Inverse Key Matrix:
[[44 35 21 35 57 17]
 [39 29 39 25 54 41]
 [20 50 57  6 11 11]
 [11 35 32 52 56  3]
 [20 25 42 40 22 47]
 [ 3  2 13 55 54 53]]
Original Text: Hello My name is Nic
Ciphertext: stYukAIJTCetRBBbfYcZ.uzF
Decrypted Text: Hello My name is NicAAAA


In [5221]:
lyrics = '''
We're no strangers to love
You know the rules and so do I
A full commitment's what I'm thinkin' of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching, but you're too shy to say it
Inside, we both know what's been going on
We know the game and we're gonna play it
And if you ask me how I'm feeling
Don't tell me you're too blind to see
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching, but you're too shy to say it
Inside, we both know what's been going on
We know the game and we're gonna play it
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
'''

In [5222]:
modulo = 59
keyMatrixSize = 8
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
print("Alphabet Length:", len(alphabet))

# plaintext = lyrics.replace("\n", " ")
plaintext = lyrics
print("Plain Text Length:", len(plaintext))
print(f"Plain Text Length / {keyMatrixSize}:", len(plaintext)/keyMatrixSize)
print(f"Plain Text Length % {keyMatrixSize}:", len(plaintext)%keyMatrixSize)
print(f"Plain Text Length % 59:", len(plaintext)%59)
print(f"Test Plaintext Length % 59 - Length %{keyMatrixSize}:", len(plaintext)%59 - len(plaintext)%keyMatrixSize)
key_matrix, inverse_key_matrix = generateKeyMatrix_nxn(keyMatrixSize, modulo)
print()


# Compute inverse key matrix dynamically
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encryptNN(plaintext, key_matrix, modulo, alphabet)

# Decrypt the ciphertext
decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print()
print(f"Ciphertext: {ciphertext}")
print()
print(f"Decrypted Text: {decrypted_text}")

Alphabet Length: 59
Plain Text Length: 1719
Plain Text Length / 8: 214.875
Plain Text Length % 8: 7
Plain Text Length % 59: 8
Test Plaintext Length % 59 - Length %8: 1

Key Matrix:
[[58 56 29 20 56 39  3  5]
 [14 58 16 54 33  5 35 38]
 [50 10 49 33 13 19 10 43]
 [47  1 47 41 44  4 55 47]
 [37 22 21 26 20 49 38 15]
 [52 42  4 54  5 12 52 13]
 [39 46 15 19  6 48  8 31]
 [37 41 18 13 56  9 13 38]]
Inverse Key Matrix:
[[40 49 10 42 47 31 39 28]
 [45 18 49 48 46 13 56 15]
 [36 19 54 22 23  7  6  2]
 [21 48 40 42 20 43  8  3]
 [48  3 47 34  9 53 16  2]
 [43 27 53 42 42 53 32 57]
 [56 14 10 22 33 24  3 22]
 [ 4 44 52 23 48 19 16 30]]
Original Text: 
We're no strangers to love
You know the rules and so do I
A full commitment's what I'm thinkin' of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gon